In [104]:
import pandas as pd
import numpy as np
import altair as alt
import copy

In [105]:
alt.data_transformers.enable('default', max_rows=None) 

DataTransformerRegistry.enable('default')

### split sata into train and test (on first 600 rows of final dataset)

In [128]:
df = pd.read_csv('data\df_final_standardized.csv')
df = df.sort_values(by = ['user_id','task_name'])
df = df[:3396]
print(len(df['user_id'].unique()))
df.head()

700


,Unnamed: 0,user_id,attempt_id,task_name,32_84_time_mean,65_78_time_mean,68_32_time_mean,69_32_time_mean,69_82_time_mean,72_69_time_mean,...,32_time_count,65_time_count,69_time_count,72_time_count,73_time_count,78_time_count,79_time_count,82_time_count,83_time_count,84_time_count
0,0,?A1OLRUT93TXWEP?,3200,copy_1,0.462054,0.390965,0.000000,0.369795,0.000000,0.305649,...,113.0,46.0,65.0,30.0,32.0,21.0,30.0,25.0,38.0,45.0
1,1,?A1OLRUT93TXWEP?,3201,copy_2,0.496368,0.259536,0.242957,0.324269,0.000000,0.252190,...,118.0,37.0,49.0,25.0,26.0,30.0,35.0,24.0,27.0,41.0
2,2,?A1OLRUT93TXWEP?,3202,fake,0.404865,0.116679,0.252171,0.282016,0.338975,0.000000,...,126.0,38.0,57.0,29.0,28.0,32.0,38.0,27.0,27.0,47.0
3,3,?A1OLRUT93TXWEP?,3203,true,0.466256,0.299536,0.215586,0.241998,0.338975,0.000000,...,136.0,54.0,85.0,32.0,44.0,30.0,41.0,35.0,51.0,56.0
4,4,A002160837SWJFPIAI7L7,0,copy_1,-0.064592,-0.135892,-0.129129,-0.098892,0.067733,-0.067355,...,96.0,13.0,34.0,25.0,24.0,25.0,34.0,10.0,24.0,12.0


In [70]:
# filter necessary columns
col_abbreviatoins = ['user_id','mean', 'var', 'std']
# col_abbreviatoins = ['32_65']
filter_col = [col for col in df.columns if any(c in col for c in col_abbreviatoins)]

In [76]:
# split into train and test datasets
train_df = df.loc[df.task_name != 'copy_2',filter_col]
test_df = df.loc[df.task_name == 'copy_2',filter_col]
train_df.head(10)

,user_id,32_84_time_mean,65_78_time_mean,68_32_time_mean,69_32_time_mean,69_82_time_mean,72_69_time_mean,73_78_time_mean,78_32_time_mean,82_69_time_mean,...,32_time_std,65_time_std,69_time_std,72_time_std,73_time_std,78_time_std,79_time_std,82_time_std,83_time_std,84_time_std
0,?A1OLRUT93TXWEP?,0.462054,0.390965,0.000000,0.369795,0.000000,0.305649,0.000000,0.441406,0.000000,...,0.031098,0.013794,0.011000,0.017290,0.018446,0.016977,0.011357,0.012109,0.016541,0.020258
2,?A1OLRUT93TXWEP?,0.404865,0.116679,0.252171,0.282016,0.338975,0.000000,0.000000,0.441406,0.362171,...,0.053173,0.013702,0.015462,0.016182,0.012599,0.010309,0.012261,0.008412,0.016226,0.020640
3,?A1OLRUT93TXWEP?,0.466256,0.299536,0.215586,0.241998,0.338975,0.000000,0.000000,0.379460,0.000000,...,0.057900,0.017078,0.010501,0.017726,0.021069,0.057156,0.018779,0.012508,0.015563,0.020913
4,A002160837SWJFPIAI7L7,-0.064592,-0.135892,-0.129129,-0.098892,0.067733,-0.067355,0.056447,0.042291,-0.133687,...,0.094073,0.223636,0.102905,0.110961,0.137848,0.178166,0.165073,0.110989,0.152581,0.126284
6,A002160837SWJFPIAI7L7,0.011371,0.088108,0.009082,-0.070699,0.042679,-0.062034,0.098005,0.173707,-0.185166,...,0.106739,0.118430,0.110039,0.139147,0.190457,0.168017,0.113844,0.095283,0.111088,0.133154
7,A002160837SWJFPIAI7L7,-0.135391,-0.108464,-0.066528,-0.071992,0.031785,-0.121540,-0.067074,-0.256234,-0.086350,...,0.135519,0.083866,0.113055,0.122549,0.156333,0.176630,0.182143,0.238225,0.137561,0.189543
8,A0436270XG2E3RS5T61O,0.122911,0.216679,0.199400,0.046827,0.000000,0.116970,0.000000,0.009548,0.076668,...,0.172034,0.134521,0.131469,0.210188,0.152906,0.183525,0.164988,0.175462,0.023965,0.083781
10,A0436270XG2E3RS5T61O,0.147675,0.120108,0.081440,0.062454,0.267079,0.176089,0.000000,0.160117,0.134360,...,0.191220,0.177162,0.125995,0.020911,0.156854,0.166340,0.164860,0.169413,0.000000,0.029130
11,A0436270XG2E3RS5T61O,0.164669,0.193504,0.252868,0.069809,0.319367,0.093385,0.343027,0.156746,0.239094,...,0.175981,0.030479,0.106859,0.041278,0.157745,0.149951,0.173551,0.150888,0.019178,0.022312
12,A10075453GKWMB0SRB9SQ,0.037469,-0.031892,0.011928,0.055351,-0.043378,-0.033974,0.301209,-0.073477,0.143236,...,0.116166,0.103742,0.110280,0.125214,0.144165,0.151929,0.119767,0.114784,0.112773,0.117904


In [77]:
# test_df.drop('predicted_user_id', inplace=True)
train_targets = train_df['user_id']
train_data = np.nan_to_num(train_df.iloc[:,1:])
test_data = np.nan_to_num(test_df.iloc[:,1:])
test_targets = test_df['user_id']

# PCA+KNN (accuracy = 81.33%)

In [87]:
# pca = PCA(n_components = 0.9)
# pca.fit(train_data,train_targets)
# print(np.round(pca.explained_variance_, 3))
# print()
# [np.sum(variances[:i])for i in np.arange(len(variances)) + 1]

In [81]:
from sklearn.decomposition import PCA
from sklearn import neighbors

pca = PCA(n_components = 0.9)
knn = neighbors.KNeighborsClassifier(n_neighbors=1, p=2)

pca.fit(train_data,train_targets)
knn.fit(pca.transform(train_data), train_targets)

acc_knn = knn.score(pca.transform(test_data), test_targets)
print(f'accuracy: {round(acc_knn * 100, 2)}%')

accuracy: 64.19%


In [64]:
predicted_pca_knn = knn.predict(pca.transform(test_data))
predicted_df_pca_knn = copy.deepcopy(test_df)

predicted_df_pca_knn['predicted_user_id'] = predicted_pca_knn
# predicted_df_pca_knn.head()

predicted_users_pca_knn = predicted_df_pca_knn[predicted_df_pca_knn['user_id'] == \
                                                predicted_df_pca_knn['predicted_user_id']][['user_id','predicted_user_id']]
print(f'number of predicted users: {len(predicted_users_pca_knn)}')
predicted_users_pca_knn.head()

number of predicted users: 513


,user_id,predicted_user_id
1,?A1OLRUT93TXWEP?,?A1OLRUT93TXWEP?
5,A002160837SWJFPIAI7L7,A002160837SWJFPIAI7L7
13,A10075453GKWMB0SRB9SQ,A10075453GKWMB0SRB9SQ
33,A10BJJH0V7T6X2,A10BJJH0V7T6X2
41,A10E8QHPV30BAD,A10E8QHPV30BAD


# KNN (accuracy = 83.33%)

In [9]:
# df = pd.read_csv('data\df_final_standardized.csv')
df.head()

,Unnamed: 0,user_id,attempt_id,task_name,32_84_time_mean,65_78_time_mean,68_32_time_mean,69_32_time_mean,69_82_time_mean,72_69_time_mean,...,32_time_count,65_time_count,69_time_count,72_time_count,73_time_count,78_time_count,79_time_count,82_time_count,83_time_count,84_time_count
0,0,?A1OLRUT93TXWEP?,3200,copy_1,0.462054,0.390965,0.000000,0.369795,0.000000,0.305649,...,113.0,46.0,65.0,30.0,32.0,21.0,30.0,25.0,38.0,45.0
1,1,?A1OLRUT93TXWEP?,3201,copy_2,0.496368,0.259536,0.242957,0.324269,0.000000,0.252190,...,118.0,37.0,49.0,25.0,26.0,30.0,35.0,24.0,27.0,41.0
2,2,?A1OLRUT93TXWEP?,3202,fake,0.404865,0.116679,0.252171,0.282016,0.338975,0.000000,...,126.0,38.0,57.0,29.0,28.0,32.0,38.0,27.0,27.0,47.0
3,3,?A1OLRUT93TXWEP?,3203,true,0.466256,0.299536,0.215586,0.241998,0.338975,0.000000,...,136.0,54.0,85.0,32.0,44.0,30.0,41.0,35.0,51.0,56.0
4,4,A002160837SWJFPIAI7L7,0,copy_1,-0.064592,-0.135892,-0.129129,-0.098892,0.067733,-0.067355,...,96.0,13.0,34.0,25.0,24.0,25.0,34.0,10.0,24.0,12.0


In [88]:
n_neighbors = 1
clf = neighbors.KNeighborsClassifier(n_neighbors, weights='uniform')

clf.fit(train_data, train_targets)
acc_clf = clf.score(test_data, test_targets)
print(f'accuracy: {round(acc_clf * 100, 2)}%')

accuracy: 83.33%


In [11]:
predicted_knn = clf.predict(test_data)
predicted_df_knn = copy.deepcopy(test_df)
predicted_df_knn['predicted_user_id'] = predicted_knn


predicted_users_knn = predicted_df_knn[predicted_df_knn['user_id'] == \
                                   predicted_df_knn['predicted_user_id']][['user_id','predicted_user_id']]
print(f'number of predicted users: {len(predicted_users_knn)}')
predicted_users_knn.head()

number of predicted users: 125


,user_id,predicted_user_id
1,?A1OLRUT93TXWEP?,?A1OLRUT93TXWEP?
5,A002160837SWJFPIAI7L7,A002160837SWJFPIAI7L7
9,A0436270XG2E3RS5T61O,A0436270XG2E3RS5T61O
13,A10075453GKWMB0SRB9SQ,A10075453GKWMB0SRB9SQ
29,A10A14T9KVM47E,A10A14T9KVM47E


In [12]:
len(predicted_users_knn['user_id'].unique())

105

# Random Forest

In [30]:
from sklearn.ensemble import RandomForestClassifier

# Create the model with 100 trees
rndf = RandomForestClassifier(n_estimators=100, 
                               bootstrap = True,
                               max_features = 'sqrt')
# Fit on training data
rndf.fit(train_data, train_targets)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='sqrt', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [31]:
rndf.score(test_data, test_targets)

0.8866666666666667